In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel(f'./file/6.url.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)
input_['SKU Count'] = input_['SKU Count'].astype(int)
input_['Page'] = input_['Page'].astype(int)
input_['No'] = input_['No'].astype(int)

input_ = input_.drop_duplicates(['Url'],
                                keep='first',
                                ignore_index=True)
input_ = input_.sort_values(by=['Url'],
                            ascending=[True],
                            ignore_index=True)

input_['No'] = [i+1 for i in range(len(input_))]

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：3460


['https://www.auveco.com/1-2-14-hex-socket-hd-pipe-plug-9285',
 'https://www.auveco.com/1-2-20-r-h-wheel-nut-chrysler-8742',
 'https://www.auveco.com/1-2-20-right-hand-wheel-nut-zinc-8744',
 'https://www.auveco.com/1-4-14-license-plate-nut-nylon-gm-11719',
 'https://www.auveco.com/1-4-18-hex-socket-hd-pipe-plug-9283',
 'https://www.auveco.com/1-4-20-free-spinning-washer-nut-5-8od-15345',
 'https://www.auveco.com/1-4-28-x-2-phillips-hdlamp-adj-screw-zinc-11246',
 'https://www.auveco.com/1-4-japanese-pipe-plug-24288',
 'https://www.auveco.com/1-8-27-hex-hd-pipe-plug-9278',
 'https://www.auveco.com/1-8-27-hex-socket-hd-pipe-plug-9282',
 'https://www.auveco.com/10-1-5-x-40mm-hex-hd-sems-29mm-wash-od-phosphate-11641',
 'https://www.auveco.com/10-1-50-x-33mm-ind-hex-wash-hd-w-dog-point-phosphate-12327',
 'https://www.auveco.com/10-x-5-8-phillips-pan-head-tap-screw-zinc-2808',
 'https://www.auveco.com/12-28-x-1-11-16-headlight-adjusting-assembly-gm-11871',
 'https://www.auveco.com/12-28-x-1-3

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(input_.loc[a, 'Url'],
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            with open('./part.html', 'w', encoding='UTF-8') as file:
                file.write(str(soup))

            if html.xpath('//h3/text()')[0].strip() == 'Can\'t find the item you are looking for on this list? Download this':
                break

            title = html.xpath('//h1[@itemprop="name"]/text()')[0].strip()
            if html.xpath('//li[@class="global-views-breadcrumb-item"]/a[@data-touchpoint="home"]/text()')[0].strip() == 'Home' and html.xpath('//li[@class="global-views-breadcrumb-item-active"]/text()')[0].strip() == title:
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1


<Response [200]>

In [6]:
html.xpath('//h3/text()')[0].strip() == 'Can\'t find the item you are looking for on this list? Download this'

False

In [7]:
list_src = html.xpath('//ul[@class="bxslider"]/li[@class="product-details-image-gallery-container"]/noscript/img[@itemprop="image"]/@src') + html.xpath('//div[@class="product-details-image-gallery"]/div[@class="product-details-image-gallery-detailed-image"]/noscript/img[@class="center-block"]/@src')
list_src = sorted(list(set(list_src)), key=list_src.index)

list_src

['https://www.auveco.com/site/item-images/9285_image-1.jpg?resizeid=3&resizeh=600&resizew=600']

In [8]:
dict_src = {}
for i, src in zip(range(len(list_src)), list_src):
    dict_src[str(i)] = src.strip()

dict_src

{'0': 'https://www.auveco.com/site/item-images/9285_image-1.jpg?resizeid=3&resizeh=600&resizew=600'}

In [9]:
sku = html.xpath('//span[@itemprop="sku"]/text()')[0].strip()

sku

'9285'

In [10]:
dict_description, list_tr = {}, html.xpath('//form[@id="product-details-full-form"]/table[@class="specs-table table table-striped"]/tbody/tr')
for i, tr in zip(range(len(list_tr)), list_tr):
    dict_description[str(i)] = {tr.xpath('./td[1]/text()')[0].strip(): tr.xpath('./td[2]/text()')[0].strip()}

dict_description

{'0': {'Pcs/Unit': '25'},
 '1': {'Head Style': 'Socket Head'},
 '2': {'Screw Size': '1/2 in.-14'},
 '3': {'Finish': 'Plain'},
 '4': {'Material': 'Cold Rolled Steel'},
 '5': {'Country': 'United States'},
 '6': {'Type': 'Pipe Plug'}}

In [11]:
list_oem = [text.strip() for text in html.xpath('//form[@id="product-details-full-form"]/div[@data-view="Additional.Detail.Fields"]/div/table[@class="specs-table table table-striped"]/tbody/tr/td[1]/text()')]
list_oem = sorted(list(set(list_oem)), key=list_oem.index)

oem = ';'.join(list_oem)

oem

'444667;444789;25522466'

In [12]:
import json

df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'SKU': sku,
                         'Title': title,
                         'OEM': oem,
                         'Picture': '',
                         'Url': input_.loc[a, 'Url'],
                         'Json_Src': json.dumps(dict_src),
                         'Json_Description': json.dumps(dict_description)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,SKU,Title,OEM,Picture,Url,Json_Src,Json_Description
0,1,9285,"1/2""-14 Hex Socket Head Pipe Plug",444667;444789;25522466,,https://www.auveco.com/1-2-14-hex-socket-hd-pi...,"{""0"": ""https://www.auveco.com/site/item-images...","{""0"": {""Pcs/Unit"": ""25""}, ""1"": {""Head Style"": ..."


In [13]:
crawler_status = 'ok'

crawler_status

'ok'

In [14]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [15]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,No,SKU,Title,OEM,Picture,Url,Json_Src,Json_Description
0,1,9285,"1/2""-14 Hex Socket Head Pipe Plug",444667;444789;25522466,,https://www.auveco.com/1-2-14-hex-socket-hd-pi...,"{""0"": ""https://www.auveco.com/site/item-images...","{""0"": {""Pcs/Unit"": ""25""}, ""1"": {""Head Style"": ..."


In [16]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
